In [21]:
from pathlib import Path

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec

from skimage.transform import resize

import tensorflow as tf


In [22]:
# Set GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Set colormap
plt.set_cmap('magma')
plt.close()

# Ignore annoying Futurewarning from `gray2rgba`
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Set ... AUTOTUNE?
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Indirectly enable autocomplete
%config Completer.use_jedi = False

In [23]:
# Get training data filepaths
%cd C:/Users/laure/OneDrive - Delft University of Technology/Cloud/TU Delft/Jaar 3/BEP/Random Mess
data_dir = Path("")
fps_src = list(data_dir.glob('../wetransfer-3c006d/_REPOSITORY/png/images_resized1024/*.png'))
fps_tgt = list(data_dir.glob('../wetransfer-3c006d/_REPOSITORY/png/masks_resized1024/*.png'))

# Get DataFrame for training
df_trn = pd.DataFrame()
df_trn['source'] = fps_src

df_trn['target']= fps_tgt



# Shuffle DataFrame
df_trn = df_trn.sample(frac=1, random_state=1234).reset_index(drop=True)

# Set source and target filepaths
fps_src = df_trn['source'].values
fps_src = fps_src.astype('str')
fps_tgt = df_trn['target'].values
fps_tgt = fps_tgt.astype('str')

C:\Users\laure\OneDrive - Delft University of Technology\Cloud\TU Delft\Jaar 3\BEP\Random Mess


In [24]:
# Choose validation size
def split(tst_size = 0.2, k = 0):
    """
This function splits the source and target data into a training set and validation set. Set size and index can be changed for cross validation. A bit janky, but it works...
    :param tst_size: Size of the validation/test data set. Defaults to 20%
    :param k: The index for k-fold cross validation.
    :return: 4 arrays, with the file paths to respectively the source (1) and target (2) data for training and the source (3) and target (4) data for validation
    """
    n_validation = int(tst_size * len(df_trn))
    n_total = len(df_trn)

    
    # Split into training and validation sets
    fps_trn_src = np.concatenate((fps_src[:n_total-n_validation*(k+1)][:], fps_src[n_total-n_validation*k:][:]))
    fps_trn_tgt = np.concatenate((fps_tgt[:n_total-n_validation*(k+1)][:], fps_tgt[n_total-n_validation*k:][:]))
    fps_val_src = fps_src[n_total-n_validation*(k+1):n_total-n_validation*k][:]
    fps_val_tgt = fps_tgt[n_total-n_validation*(k+1):n_total-n_validation*k][:]
    return fps_trn_src, fps_trn_tgt, fps_val_src, fps_val_tgt

fps_trn_src, fps_trn_tgt, fps_val_src, fps_val_tgt = split(tst_size = 0.2, k = 2)

In [40]:
def load_images(fp_src, fp_tgt, size = (256, 256)):
    """
    Loads the images from the file paths
    :param fp_src: file path to source image
    :param fp_tgt: file path to target image
    :param size: the size to which the images should be resized
    :return: two image arrays
    """
    img_src = tf.io.decode_image(tf.io.read_file(fp_src),
                                dtype='float32',
                                expand_animations=False)
    img_tgt = tf.io.decode_image(tf.io.read_file(fp_tgt),
                                dtype='float32',
                                expand_animations=False)
    #img_src = tf.image.resize(img_src, size= size)
    #img_tgt = tf.image.resize(img_tgt, size= size)
    

    return img_src, img_tgt

In [37]:
def augment_images(img_src, img_tgt):
    """
    A function to augment the images by rotating, flipping and changing contrast+brightness
    :param img_src: The source image
    :param img_tgt: The target image
    :return: Two possibly augmented image
    """
    # rotate
    k = tf.random.uniform([], 0, 4, dtype=tf.int32)
    img_src_aug = tf.image.rot90(img_src, k)
    img_tgt_aug = tf.image.rot90(img_tgt, k)
    
    # flip left/right
    k = tf.random.uniform([], 0, 1, dtype=tf.float32)
    img_src_aug = tf.cond(k>0.5, lambda: tf.image.flip_left_right(img_src_aug), lambda: img_src_aug)
    img_tgt_aug = tf.cond(k>0.5, lambda: tf.image.flip_left_right(img_tgt_aug), lambda: img_tgt_aug)
    
    # flip up/down
    k = tf.random.uniform([], 0, 1, dtype=tf.float32)
    img_src_aug = tf.cond(k>0.5, lambda: tf.image.flip_up_down(img_src_aug), lambda: img_src_aug)
    img_tgt_aug = tf.cond(k>0.5, lambda: tf.image.flip_up_down(img_tgt_aug), lambda: img_tgt_aug)


    # The augmentations below are only applied to the source image
    # Contrast
    u = tf.random.uniform([], 0, 1, dtype=tf.float32)
    img_src_aug = tf.cond(u > 0.5, lambda: img_src_aug,
                             lambda: tf.image.random_contrast(img_src_aug, 0.75, 1.5))
    # Brightness
    u = tf.random.uniform([], 0, 1, dtype=tf.float32)
    img_src_aug = tf.cond(u > 0.5, lambda: img_src_aug,
                               lambda: tf.image.random_brightness(img_src_aug, 0.2))

    # clip to 0,1
    img_src_aug = tf.clip_by_value(img_src_aug, clip_value_min=0, clip_value_max=1)
    
    # swap RGB to BGR
#     k = tf.random.uniform([], 0, 1, dtype=tf.float32)
#     img_src_aug = tf.cond(k>0.5, lambda: tf.reverse(img_src_aug, axis=[-1]), lambda: img_src_aug)
    
    return img_src_aug, img_tgt_aug

In [38]:
def create_dataset(fps_src, fps_tgt, batch_size=16, size = None, augment = True, repeat = 4):
    """
A quick function to create a dataset with the images from a list of file paths
    """
    # Create dataset of filepaths
    ds_fps = tf.data.Dataset.from_tensor_slices((fps_src, fps_tgt))

    # Repeat
    if repeat:
        ds_fps = ds_fps.repeat(count=repeat)

    # Convert dataset of filepaths into a dataset of images
    ds = ds_fps.map(lambda x, y: load_images(x, y, size))

    # Augment
    if augment:
        ds = ds.map(lambda x, y: augment_images(x, y))

    # Batch
    ds = ds.batch(batch_size)

    # Prefetch
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [39]:
# set some training parameters
batch_size = 1      # functions get_n_tensors and get_max_batch_size from clemnet can be used to determine max batch size
SIZE = 1024
IMG_CHANNELS = 3

# create training, validation, testing datasets
ds_trn = create_dataset(fps_trn_src, fps_trn_tgt,
                        batch_size=batch_size, 
                        size = (SIZE,SIZE),
                       repeat = 16)

ds_val = create_dataset(fps_val_src, fps_val_tgt,
                        batch_size=batch_size, 
                        size = (SIZE,SIZE), augment = True,
                       repeat = 16)

ds_test = create_dataset(fps_val_src, fps_val_tgt,
                        batch_size=batch_size, 
                        size = (SIZE,SIZE), augment = False, repeat = False)

# create a dataset with all images to also look at performances on training data
ds_all = create_dataset(fps_src, fps_tgt,
                        batch_size=batch_size, 
                        size = (SIZE,SIZE), augment = False, repeat = False)


<PrefetchDataset shapes: ((None, None, None, None), (None, None, None, None)), types: (tf.float32, tf.float32)>

In [45]:
# define model if using a ResNet backbone
# %env SM_FRAMEWORK=tf.keras
# import segmentation_models as sm
#
# BACKBONE = 'resnet34'
# preprocess_input = sm.get_preprocessing(BACKBONE)
#
#
#
# model = sm.Unet(BACKBONE, encoder_weights='imagenet')
# model.summary()

env: SM_FRAMEWORK=tf.keras
Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, None, None,  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_34 (ZeroPadding2 (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_34[0][0]          
___________________________________________________________

In [ ]:
#Build the model

inputs = tf.keras.layers.Input((SIZE, SIZE, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.summary()

In [43]:
# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])


checkpoint_path = "training_1/cp.h5"
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=30, 
                                         monitor='val_loss',
                                        min_delta = 0,
                                        verbose = 1),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                          verbose=1,
                                          save_best_only=True, 
                                          save_freq='epoch')] 


In [16]:
## If required, training can be resumed from a previously trained model.
#model = tf.keras.models.load_model('training_1/cp.h5')

In [44]:
history = model.fit(ds_trn,
                    validation_data=ds_val,
                    epochs=600,
                    callbacks=callbacks)

Epoch 1/600


ResourceExhaustedError:  OOM when allocating tensor with shape[64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_11/decoder_stage2b_bn/FusedBatchNormV3 (defined at <ipython-input-44-d3c3469f5abf>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_23610]

Function call stack:
train_function


In [ ]:
# Collect as DataFrame
df = pd.DataFrame(history.history)
cols = np.stack(df.columns.values.reshape(2, -1), axis=1)

# Plot training data
ncols = len(cols)
fig, axes = plt.subplots(ncols=ncols, figsize=(5*ncols, 4))
for i in range(ncols):
    df.plot(y=cols[i], marker='o', mfc='none', ax=axes[i])
[ax.grid(ls=':') for ax in axes];

In [ ]:
# save the model
model.save('Final/16_23_maskUnet_1024v2.h5')


In [41]:
def metrics(truth,prediction):
    """
    A function to return the IoU and PCC metrics from a given ground truth and prediction
    """
    threshold = 0.1
  
    truth = np.float32(truth)

    
    prediction = prediction > threshold

    
    intersection = np.logical_and(truth,prediction)
    union = np.logical_or(truth,prediction)
    iou = np.sum(intersection)/np.sum(union)

    FP = np.logical_and(np.logical_not(truth),prediction)
    FN = np.logical_and(truth,np.logical_not(prediction))
    TP = np.logical_and(truth,prediction)
    TN = np.logical_and(np.logical_not(truth),np.logical_not(prediction))

    
    pearson = np.corrcoef(truth.ravel(), prediction.ravel())[0,1]
    return round(iou,2), round(pearson,2)

In [ ]:
# Loop through a batch size worth of images
model = tf.keras.models.load_model('Final/24_31_maskUNet_1024.h5', compile=False)


dataset = ds_test

iterations = iter(dataset)
for j in tqdm(range(int(len(dataset)/batch_size))):
    # Make predictions
    
    batch_src, batch_tgt = next(iterations)
    predictions = model.predict(batch_src,
                                verbose=1)
    
    
    ncols = 4
    nrows = int(np.ceil(batch_size / ncols))
    fig = plt.figure(figsize=(ncols*9, nrows*3))
    gs = gridspec.GridSpec(nrows=nrows, ncols=ncols, figure=fig,
                           hspace=0.08, wspace=0.05)

    # Loop through a batch size worth of images
    for i in range(batch_size):

        # Fetch images from batch
        img_src = batch_src[i,:,:,:].numpy().astype(np.float32)
        img_tgt = batch_tgt[i,:,:,0].numpy().astype(np.float32)
        img_prediction= predictions[i,:,:,:].astype(np.float32)
        # Upsample to fit src
        img_tgt = resize(img_tgt, img_src.shape)


        # Set up mini gridspec
        gss = gridspec.GridSpecFromSubplotSpec(nrows=1, ncols=3, wspace=0,
                                               subplot_spec=gs[i])
        ax1 = fig.add_subplot(gss[0])
        ax2 = fig.add_subplot(gss[1])
        ax3 = fig.add_subplot(gss[2])

        # Plot images
        ax1.imshow(img_src)
        ax2.imshow(img_tgt, cmap='magma', vmin=0, vmax=1)
        ax3.imshow(img_prediction)
        

        # Add metric
        ax3.text(0.1, 0.9, 'iou ' + str(metrics(img_tgt[:,:,0], img_prediction[:,:,0])[0]), fontsize=12, transform=ax3.transAxes, c='yellow')
        ax3.text(0.1, 0.8, 'prsn ' + str(metrics(img_tgt[:,:,0], img_prediction[:,:,0])[1]), fontsize=12, transform=ax3.transAxes, c='yellow')


        # Aesthetics
        [ax.axis('off') for ax in [ax1, ax2, ax3]]

